## Pre-processing

In [1]:
import pandas as pd
df = pd.read_csv('QVI_transaction_data.csv')
df.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,17-10-2018,1.0,1000.0,1.0,5.0,Natural Chip Compny SeaSalt175g,2.0,6.0
1,14-05-2019,1.0,1307.0,348.0,66.0,CCs Nacho Cheese 175g,3.0,6.3
2,20-05-2019,1.0,1343.0,383.0,61.0,Smiths Crinkle Cut Chips Chicken 170g,2.0,2.9
3,17-08-2018,2.0,2373.0,974.0,69.0,Smiths Chip Thinly S/Cream&Onion 175g,5.0,15.0
4,18-08-2018,2.0,2426.0,1038.0,108.0,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3.0,13.8


#### Creating 2 new features: WEIGHT & PACK_SIZE 

In [ ]:
df['WEIGHT'] = df['PROD_NAME'].str.extract(r'(\d+)')
df['WEIGHT'] = pd.to_numeric(df['WEIGHT'], errors='coerce')  # Converts to float, NaN for non-numeric values
weight = df['WEIGHT']

def classify_pack_size(weight):
    if 70 <= weight <= 100:
        return 'Small'
    elif 101 <= weight <= 200:
        return 'Medium'
    elif 201 <= weight <= 300:
        return 'Large'
    elif 301 <= weight <= 380:
        return 'Extra-Large'
    else:
        return 'Unknown' 

df['PACK_SIZE'] = df['WEIGHT'].apply(classify_pack_size)
df.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,WEIGHT,PACK_SIZE
0,17-10-2018,1.0,1000.0,1.0,5.0,Natural Chip Compny SeaSalt175g,2.0,6.0,175.0,Medium
1,14-05-2019,1.0,1307.0,348.0,66.0,CCs Nacho Cheese 175g,3.0,6.3,175.0,Medium
2,20-05-2019,1.0,1343.0,383.0,61.0,Smiths Crinkle Cut Chips Chicken 170g,2.0,2.9,170.0,Medium
3,17-08-2018,2.0,2373.0,974.0,69.0,Smiths Chip Thinly S/Cream&Onion 175g,5.0,15.0,175.0,Medium
4,18-08-2018,2.0,2426.0,1038.0,108.0,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3.0,13.8,150.0,Medium


#### Creating a new feature BRAND by extracting the brand name from PROD_NAME

In [ ]:
df["PROD_NAME"] = df["PROD_NAME"].astype(str).fillna("")
# Predefined list of known chip brands
known_brands = [
    "Smiths", "Doritos", "Kettle", "CCs", "Grain Waves", "Twisties", "Cheezels",
    "WW", "Thins", "Natural Chip Company", "Old El Paso", "Burger Rings",
    "Cheetos", "Tostitos", "Lay’s", "Pringles", "Ruffles", "Sun Chips", "Takis",
    "Herr’s", "Wise", "Cape Cod", "Utz", "Zapp’s", "Popchips", "Fritos", "Munchos", "Tyrrells",
    "Kettle Brand", "Miss Vickie’s", "Stacy’s", "Terra Chips", "Woolworths","Infuzions","Twisties","Grain Waves"
]
names=df["PROD_NAME"]

def extract_brand(names):
    words = names.split()
    for brand in known_brands:
        if names.startswith(brand):
            return brand
    return words[0]  # If unknown, we assume that the first word is the brand name

df["BRAND"] = df["PROD_NAME"].apply(extract_brand)

# Identify new brands (not in predefined list)
new_brands = df[~df["BRAND"].isin(known_brands)]["BRAND"].unique()

# Saving results
df.to_csv("QVI_transaction_data_final1.csv", index=False)

# Print new brands just to review
print("Potential new brands found:", new_brands)

Potential new brands found: ['Natural' 'NCC' 'Infzns' 'Red' 'Dorito' 'Smith' 'GrnWves' 'Cobs' 'French'
 'RRD' 'Snbts' 'Sunbites' 'nan']


In the above output, we can see that there are 2 names referring to the same company in some cases for example: Snbts and Sunbites are the same company. 
Other such cases: Red Rock Deli and RRD, Infzns and Infuzions, etc...
We fixed those cases using filter & find and replace in excel.

#### Joining the transaction & customer data

In [9]:
df_purchase = pd.read_csv('QVI_purchase_behaviour.csv')
df_merged = pd.merge(df,df_purchase,how='inner',on='LYLTY_CARD_NBR')
df_merged.to_csv('QVI_data_joined_1.csv',index=False)